In [1]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
import numpy as np
import re
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import random
import time
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from tqdm import tqdm
import wandb
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from IPython.display import display

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
WANDB_API_KEY = '27687968cab05dfe2e7fe4fb3b59772e1f7c167a'
wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

## Preprocessing

In [3]:
def preprocess(data):
    data = re.sub(r'\n|\s+', ' ', data) #newline and multiple spaces -> single space
    data = re.sub(r'[’‘]', '\'', data) #apostrophes
    data = re.sub(r'[“”`\' ]|[–—-]', ' ', data) #quotes and dashes
    data = re.sub(r'(?<!\w)([.!?])(?!\w)', r' \1 ', data) #dont remove punctuation
    data = re.sub(r'[™•]', ' ', data) #remove other unwanted symbols
    return data.strip() #strip extra spaces

## Tokenization

In [4]:
def tokenize(data, min_length_sentences=6):
    sentences = sent_tokenize(data)
    sentences = [sentence for sentence in sentences if len(sentence.split()) >= min_length_sentences]
    print("Length of sentences after filtering:", len(sentences))

    words_sentences = []

    for sentence in sentences:
        words = word_tokenize(sentence)
        
        words = [word.lower() for word in words if word.lower() not in ['.', ',', '!', '?', ';', ':']]
        words = ['<s>'] + words + ['</s>']
        words_sentences.append(words)
    
    return sentences, words_sentences

## Data Preparation

In [5]:
def train_val_test_split(sentences, train_ratio=0.7, val_ratio=0.2, seed=None, num_shuffles=1):
    if seed is not None:
        random.seed(seed)
    
    for _ in range(num_shuffles):
        random.shuffle(sentences)
    
    total_sentences = len(sentences)
    
    train_size = int(total_sentences * train_ratio)
    val_size = int(total_sentences * val_ratio)
    test_size = total_sentences - train_size - val_size  # Remaining for test
    
    train_sentences = sentences[:train_size]
    val_sentences = sentences[train_size:train_size + val_size]
    test_sentences = sentences[train_size + val_size:]
    
    return train_sentences, val_sentences, test_sentences


## Loading Glove Embeddings

In [6]:
def create_glove_embeddings(glove_path):
    glove = {}
    embedding_dim = 0

    with open(glove_path, 'r') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = torch.tensor([float(val) for val in values[1:]])
            glove[word] = vector
            embedding_dim = len(values[1:])

    glove['<UNK>'] = torch.mean(torch.stack(list(glove.values())), dim=0)
    glove['<s>'] = torch.rand(embedding_dim)
    glove['</s>'] = torch.rand(embedding_dim)

    return glove

## Creation of Vocab and Encodings

In [7]:
def create_embeddings_and_encode(train_sentences, val_sentences, test_sentences, glove):
    embedding_dim = len(list(glove.values())[0])
    vocab = set()

    # create vocab from train
    vocab.update(['<s>', '</s>', '<UNK>',])

    for sentence in train_sentences:
        for word in sentence:
            if word in glove:
                vocab.add(word)
            else:
                sentence[sentence.index(word)] = '<UNK>'
                
    embeddings = np.zeros((len(vocab), embedding_dim))
    word_to_idx = {word: idx for idx, word in enumerate(vocab)}

    for word in vocab:
        if word in glove:
            embeddings[word_to_idx[word]] = glove[word]
        else:
            embeddings[word_to_idx[word]] = np.random.rand(embedding_dim)  # Random for unknown words

    def encode_sentences(sentences, word_to_idx):
        encoded_sentences = []
        for sentence in sentences:
            encoded_sentence = [word_to_idx[word] if word in word_to_idx else word_to_idx['<UNK>'] for word in sentence]
            encoded_sentences.append(encoded_sentence)
        return encoded_sentences

    encoded_train_sentences = encode_sentences(train_sentences, word_to_idx)
    encoded_val_sentences = encode_sentences(val_sentences, word_to_idx)
    encoded_test_sentences = encode_sentences(test_sentences, word_to_idx)

    return torch.FloatTensor(embeddings), encoded_train_sentences, encoded_val_sentences, encoded_test_sentences, word_to_idx, list(vocab)


## Dataset for Training NNLM

In [8]:
class NGramDataset(Dataset):
    def __init__(self, data, embeddings, n=5):
        self.n = n
        self.ngrams = []
        self.labels = []
        self.embeddings = embeddings

        for sentence in data:
            for i in range(len(sentence) - self.n):
                context_indices = sentence[i:i + self.n]
                target_index = sentence[i + self.n]

                context_embeddings = torch.cat([self.embeddings[idx] for idx in context_indices], dim=0).float()
                
                self.ngrams.append(context_embeddings)
                self.labels.append(target_index)

    def __len__(self):
        return len(self.ngrams)

    def __getitem__(self, idx):
        return self.ngrams[idx], torch.tensor(self.labels[idx], dtype=torch.long)  # Convert labels to Long



## NNLM Model

In [9]:
class NNLM(nn.Module):
    def __init__(self, embeddings, hidden_dims, n_gram=5, dropout=0.5):
        super(NNLM, self).__init__()

        self.vocab_size = embeddings.shape[0]
        self.embeddings_dim = embeddings.shape[1]

        self.embeddings = nn.Embedding.from_pretrained(embeddings, freeze=True)
        self.fc1 = nn.Linear((self.embeddings_dim) * n_gram, hidden_dims[0])
        self.fc2 = nn.Linear(hidden_dims[0], hidden_dims[1])
        self.fc3 = nn.Linear(hidden_dims[1], self.vocab_size)

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p = dropout)

    def forward(self, x):
        x = x.view(x.shape[0], -1)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.dropout(self.relu(self.fc2(x)))
        return self.fc3(x)


## Model Testing

In [10]:
def test_model(model, eval_loader, criterion):
    model.eval()
    total_loss = 0

    with torch.no_grad():
        for i, (x, y) in enumerate(tqdm(eval_loader)):
            x, y = x.float().to(device), y.type(torch.LongTensor).to(device)
            outputs = model(x)
            loss = criterion(outputs, y)
            total_loss += loss.item()
            
        avg_loss = total_loss / len(eval_loader)
        perplexity = torch.exp(torch.tensor(avg_loss))

    return avg_loss, perplexity


## Model Training

In [14]:
def train_model(model, train_loader, val_loader, optimizer, criterion, num_epochs, patience=2):
    model.to(device)
    early_stopping_counter = 0
    best_val_loss = float('inf')
    
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0

        train_data = tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}', unit='batch')

        for x, y in train_data:
            x, y = x.float().to(device), y.type(torch.LongTensor).to(device)  # Ensure Float type

            optimizer.zero_grad()
            y_pred = model(x)
            loss = criterion(y_pred, y)
            total_loss += loss.item()

            loss.backward()
            optimizer.step()

        avg_train_loss = total_loss / len(train_loader)
        perplexity = torch.exp(torch.tensor(avg_train_loss))

        print(f'Train Loss: {avg_train_loss:.4f}')
        print(f'Train Perplexity: {perplexity:.4f}')

        avg_val_loss, val_perplexity = test_model(model, val_loader, criterion)
        print(f'Val Loss: {avg_val_loss:.4f}')
        print(f'Val Perplexity: {val_perplexity:.4f}')

        # check for early stopping
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            early_stopping_counter = 0
            torch.save(model.state_dict(), '2021101072_LM1.pt')
        else:
            early_stopping_counter += 1
            if early_stopping_counter >= patience:
                print(f'Early stopping at epoch {epoch + 1}')
                break

    return model


## Save Perplexities in files

In [15]:
def save_perplexities(model, sentences, embeddings, criterion, filename, idx_to_word, n=5):
    model.eval()
    total_loss = 0
    all_sentences = []
    perplexity_scores = []

    with torch.no_grad():
        for sentence in sentences:
            sentence_loss = 0
            sentence_length = 0
            ngrams = []
            targets = []

            for i in range(len(sentence) - n):
                context_indices = sentence[i:i + n]
                target_index = sentence[i + n]

                context_embeddings = torch.cat([embeddings[idx] for idx in context_indices], dim=0).float()

                ngrams.append(context_embeddings)
                targets.append(target_index)

            for j in range(len(ngrams)):
                ngram = ngrams[j].to(device) 
                target_word = torch.tensor(targets[j], dtype=torch.long).to(device)  

                outputs = model(ngram.unsqueeze(0))  
                loss = criterion(outputs, target_word.unsqueeze(0))  
                sentence_loss += loss.item()
                sentence_length += 1

            avg_loss_per_sentence = sentence_loss / sentence_length
            sentence_perplexity = torch.exp(torch.tensor(avg_loss_per_sentence)).item()
            perplexity_scores.append(sentence_perplexity)

            sentence_words = [idx_to_word[idx] for idx in sentence]
            full_sentence = " ".join(sentence_words)
            all_sentences.append(full_sentence)

        avg_perplexity = sum(perplexity_scores) / len(perplexity_scores)

    with open(filename, 'w') as f:
        for i, sentence in enumerate(all_sentences):
            f.write(f"{sentence}\t{perplexity_scores[i]}\n")
        
        f.write(f"Average\t{avg_perplexity}\n")

    return avg_perplexity


In [16]:
def save_model(model, path):
    torch.save(model.state_dict(), path)
    print(f"Model saved to {path}")


## Running the Model

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [18]:
with open('/kaggle/input/auguste-maquet/Auguste_Maquet.txt', 'r') as f:
    corpus = f.read()

corpus = preprocess(corpus)
min_length_sentences = 6

sentences, word_sentences = tokenize(corpus)

train_sentences, val_sentences, test_sentences = train_val_test_split(word_sentences)

print("Train size:", len(train_sentences))
print("Validation size:", len(val_sentences))
print("Test size:", len(test_sentences))

Length of sentences after filtering: 45102
Train size: 31571
Validation size: 9020
Test size: 4511


In [19]:
glove = create_glove_embeddings('/kaggle/input/glove-300/glove.6B.300d.txt')

embeddings, encoded_train, encoded_val, encoded_test, word_to_idx, vocab = create_embeddings_and_encode(train_sentences, val_sentences, test_sentences, glove)


In [20]:
train_dataset = NGramDataset(encoded_train, embeddings)
val_dataset = NGramDataset(encoded_val, embeddings)
test_dataset = NGramDataset(encoded_test, embeddings)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

print(f'Train dataset size: {len(train_dataset)}')
print(f'Validation dataset size: {len(val_dataset)}')
print(f'Test dataset size: {len(test_dataset)}')

print(train_dataset[0][0].shape)

Train dataset size: 543593
Validation dataset size: 157712
Test dataset size: 77633
torch.Size([1500])


In [21]:
learning_rate = 0.01
model = NNLM(embeddings, [300, 300], 5, 0.1)
optimizer = optim.SGD(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()


In [22]:
model = train_model(model, train_loader, val_loader, optimizer, criterion, num_epochs=10)

Epoch 1/10: 100%|██████████| 8494/8494 [00:21<00:00, 392.77batch/s]


Train Loss: 6.7636
Train Perplexity: 865.7474


100%|██████████| 2465/2465 [00:03<00:00, 677.13it/s]


Val Loss: 6.2277
Val Perplexity: 506.5939


Epoch 2/10: 100%|██████████| 8494/8494 [00:21<00:00, 401.81batch/s]


...

Val Loss: 5.4025
Val Perplexity: 221.9567


Epoch 10/10: 100%|██████████| 8494/8494 [00:20<00:00, 404.99batch/s]


Train Loss: 5.4197
Train Perplexity: 225.8150


100%|██████████| 2465/2465 [00:03<00:00, 672.43it/s]

Val Loss: 5.3616
Val Perplexity: 213.0745


## Perplexity Scores

In [23]:
loss, perplexity = test_model(model, train_loader, criterion)
print(f'\nTrain Loss: {loss}')
print(f'Train Perplexity: {perplexity}')

100%|██████████| 8494/8494 [00:13<00:00, 633.66it/s]


Train Loss: 5.323368948657344
Train Perplexity: 205.07362365722656


In [24]:
loss, perplexity = test_model(model, val_loader, criterion)
print(f'\nVal Loss: {loss}')
print(f'Val Perplexity: {perplexity}')

100%|██████████| 2465/2465 [00:03<00:00, 633.28it/s]


Val Loss: 5.361691077582493
Val Perplexity: 213.08497619628906


In [25]:
loss, perplexity = test_model(model, test_loader, criterion)
print(f'\nTest Loss: {loss}')
print(f'Test Perplexity: {perplexity}')

100%|██████████| 1214/1214 [00:01<00:00, 622.98it/s]


Test Loss: 5.343334847659216
Test Perplexity: 209.20919799804688


In [26]:
save_perplexities(model, encoded_train, embeddings, criterion, '2021101072_LM1_train_perplexity.txt', vocab)
save_perplexities(model, encoded_val, embeddings, criterion, '2021101072_LM1_val_perplexity.txt', vocab)
save_perplexities(model, encoded_test, embeddings, criterion, '2021101072_LM1_test_perplexity.txt', vocab)

264.90034703705214

In [33]:
# save_model(model, '2021101072_LM1.pt')

In [34]:
import pickle

with open('data_store_nnlm.pkl', 'wb') as f:
    pickle.dump({
        'embeddings': embeddings,
        'vocab': vocab,
        'word_to_idx': word_to_idx,
        'encoded_train': encoded_train,
        'encoded_val': encoded_val,
        'encoded_test': encoded_test,
    }, f)

print("Data saved successfully!")

Data saved successfully!


## Hyperparameter Tuning

In [19]:
results = []

dropout_rates = [0.1, 0.3, 0.5]
hidden_dims_options = [[100, 100], [200, 200], [300, 300]]
learning_rates = [0.001, 0.01]
optimizers = [optim.Adam, optim.SGD]

In [20]:
for dropout in dropout_rates:
    for hidden_dims in hidden_dims_options:
        for lr in learning_rates:
            for optimizer_class in optimizers:

                wandb.init(project='hyperparameter_tuning_nnlm_a1', config={
                    'dropout': dropout,
                    'hidden_dims': hidden_dims,
                    'learning_rate': lr,
                    'optimizer': optimizer_class.__name__
                })

                model = NNLM(embeddings, hidden_dims, n_gram=5, dropout=dropout)
                optimizer = optimizer_class(model.parameters(), lr=lr)
                criterion = nn.CrossEntropyLoss()

                model = train_model(model, train_loader, val_loader, optimizer, criterion, num_epochs=10)

                avg_train_loss, train_perplexity = test_model(model, train_loader, criterion)
                avg_val_loss, val_perplexity = test_model(model, val_loader, criterion)
                avg_test_loss, test_perplexity = test_model(model, test_loader, criterion)

                wandb.log({
                    'train_loss': avg_train_loss,
                    'train_perplexity': train_perplexity,
                    'val_loss': avg_val_loss,
                    'val_perplexity': val_perplexity,
                    'test_loss': avg_test_loss,
                    'test_perplexity': test_perplexity
                })

                results.append({
                    'dropout': dropout,
                    'hidden_dims': hidden_dims,
                    'learning_rate': lr,
                    'optimizer': optimizer_class.__name__,
                    'train_loss': avg_train_loss,
                    'train_perplexity': train_perplexity,
                    'val_loss': avg_val_loss,
                    'val_perplexity': val_perplexity,
                    'test_loss': avg_test_loss,
                    'test_perplexity': test_perplexity
                })

                wandb.finish()

wandb: Currently logged in as: ashnadua (ashna-dua). Use `wandb login --relogin` to force relogin


Epoch 1/10: 100%|██████████| 8533/8533 [00:22<00:00, 385.74batch/s]


Train Loss: 5.9378
Train Perplexity: 379.1139


100%|██████████| 2428/2428 [00:03<00:00, 765.58it/s]


Val Loss: 5.5288
Val Perplexity: 251.8535


...

Train Loss: 5.7015
Train Perplexity: 299.3176


100%|██████████| 2428/2428 [00:03<00:00, 679.04it/s]


Val Loss: 5.4805
Val Perplexity: 239.9600


100%|██████████| 1212/1212 [00:01<00:00, 641.43it/s]


test_loss,▁
test_perplexity,▁
train_loss,▁
train_perplexity,▁
val_loss,▁
val_perplexity,▁
test_loss,5.51914
test_perplexity,249.42097
train_loss,5.50115
train_perplexity,244.97247
val_loss,5.47999


In [24]:
# Convert tensors to floats
for result in results:
    result['train_perplexity'] = result['train_perplexity'].item()  # Convert tensor to float
    result['val_perplexity'] = result['val_perplexity'].item()
    result['test_perplexity'] = result['test_perplexity'].item()


In [27]:
results_df = pd.DataFrame(results)

results_df['hyperparam_combination'] = results_df.apply(
    lambda row: f"Dropout: {row['dropout']}, LR: {row['learning_rate']}, HD: {row['hidden_dims']}, Opt: {row['optimizer']}", axis=1)

In [28]:
best_val_perplexity_row = results_df.loc[results_df['val_perplexity'].idxmin()]
best_test_perplexity_row = results_df.loc[results_df['test_perplexity'].idxmin()]

best_val_perplexity_df = pd.DataFrame([best_val_perplexity_row])
best_test_perplexity_df = pd.DataFrame([best_test_perplexity_row])


In [29]:
print("Best Hyperparameters with Lowest Validation Perplexity:")
display(best_val_perplexity_df)

Best Hyperparameters with Lowest Validation Perplexity:


,dropout,hidden_dims,learning_rate,optimizer,train_loss,train_perplexity,val_loss,val_perplexity,test_loss,test_perplexity,hyperparam_combination
11,0.1,"[300, 300]",0.01,SGD,5.325233,205.456314,5.341418,208.80864,5.378655,216.73056,"Dropout: 0.1, LR: 0.01, HD: [300, 300], Opt: SGD"


In [30]:
print("\nBest Hyperparameters with Lowest Test Perplexity:")
display(best_test_perplexity_df)


Best Hyperparameters with Lowest Test Perplexity:


,dropout,hidden_dims,learning_rate,optimizer,train_loss,train_perplexity,val_loss,val_perplexity,test_loss,test_perplexity,hyperparam_combination
11,0.1,"[300, 300]",0.01,SGD,5.325233,205.456314,5.341418,208.80864,5.378655,216.73056,"Dropout: 0.1, LR: 0.01, HD: [300, 300], Opt: SGD"


In [31]:
import plotly.graph_objects as go

# Create an interactive line plot
fig = go.Figure()

# Plot Train Perplexity
fig.add_trace(go.Scatter(
    x=results_df['hyperparam_combination'],
    y=results_df['train_perplexity'],
    mode='lines+markers',
    name='Train Perplexity',
    marker=dict(size=8),
    line=dict(width=2)
))

# Plot Validation Perplexity
fig.add_trace(go.Scatter(
    x=results_df['hyperparam_combination'],
    y=results_df['val_perplexity'],
    mode='lines+markers',
    name='Validation Perplexity',
    marker=dict(size=8),
    line=dict(width=2)
))

# Plot Test Perplexity
fig.add_trace(go.Scatter(
    x=results_df['hyperparam_combination'],
    y=results_df['test_perplexity'],
    mode='lines+markers',
    name='Test Perplexity',
    marker=dict(size=8),
    line=dict(width=2)
))

# Update layout with increased size
fig.update_layout(
    title='Perplexity vs. Hyperparameters',
    xaxis_title='Hyperparameters',
    yaxis_title='Perplexity',
    xaxis_tickangle=-90,
    legend_title='Perplexity Type',
    template='plotly_white',
    width=1400,  # Increase width
    height=1000,   # Increase height
    xaxis_tickfont=dict(size=10),  # Smaller x-axis tick font size
)

# Show plot
fig.show()


In [32]:
import plotly.express as px

# Melt the dataframe for easier plotting with Plotly Express
melted_df = results_df.melt(id_vars='hyperparam_combination', 
                            value_vars=['train_perplexity', 'val_perplexity', 'test_perplexity'], 
                            var_name='Perplexity Type', value_name='Perplexity')

# Create an interactive bar plot
fig_bar = px.bar(melted_df, 
                 x='hyperparam_combination', 
                 y='Perplexity', 
                 color='Perplexity Type', 
                 barmode='group',
                 color_discrete_sequence=px.colors.qualitative.D3,  # Use different colors
                 labels={'hyperparam_combination': 'Hyperparameter Combinations', 'Perplexity': 'Perplexity'},
                 title='Perplexity vs. Hyperparameter Combinations')

# Update layout
fig_bar.update_layout(
    xaxis_tickangle=-90,
    template='plotly_white',
    width=1400,  # Increase width
    height=800,   # Increase height
    xaxis_tickfont=dict(size=8),  # Smaller x-axis tick font size
    bargap=0.2  # Increase space between bars
)

# Show plot
fig_bar.show()


In [26]:
results_df.to_csv('hyperparameter_tuning_results_nnlm.csv', index=False)

In [32]:
results_df

,dropout,hidden_dims,learning_rate,optimizer,train_loss,train_perplexity,val_loss,val_perplexity,test_loss,test_perplexity,hyperparam_combination
0,0.1,"[100, 100]",0.001,Adam,5.128179,168.709549,5.513472,248.010742,5.556505,258.916260,"Dropout: 0.1, LR: 0.001, HD: [100, 100], Opt: ..."
1,0.1,"[100, 100]",0.001,SGD,6.335142,564.049866,6.275821,531.562500,6.301206,545.229126,"Dropout: 0.1, LR: 0.001, HD: [100, 100], Opt: SGD"
2,0.1,"[100, 100]",0.010,Adam,6.322811,557.136963,6.268852,527.871155,6.302869,546.136169,"Dropout: 0.1, LR: 0.01, HD: [100, 100], Opt: Adam"
3,0.1,"[100, 100]",0.010,SGD,5.465257,236.336517,5.452482,233.336533,5.492457,242.853256,"Dropout: 0.1, LR: 0.01, HD: [100, 100], Opt: SGD"
4,0.1,"[200, 200]",0.001,Adam,5.094634,163.144135,5.517128,248.919235,5.556593,258.939117,"Dropout: 0.1, LR: 0.001, HD: [200, 200], Opt: ..."
5,0.1,"[200, 200]",0.001,SGD,6.258859,522.622131,6.198226,491.875427,6.222934,504.180450,"Dropout: 0.1, LR: 0.001, HD: [200, 200], Opt: SGD"
6,0.1,"[200, 200]",0.010,Adam,6.372320,585.414246,6.333749,563.264465,6.359728,578.088989,"Dropout: 0.1, LR: 0.01, HD: [200, 200], Opt: Adam"
7,0.1,"[200, 200]",0.010,SGD,5.376096,216.176727,5.372719,215.447937,5.413367,224.385880,"Dropout: 0.1, LR: 0.01, HD: [200, 200], Opt: SGD"
8,0.1,"[300, 300]",0.001,Adam,5.110413,165.738724,5.535922,253.641556,5.569774,262.374725,"Dropout: 0.1, LR: 0.001, HD: [300, 300], Opt: ..."
9,0.1,"[300, 300]",0.001,SGD,6.229542,507.523346,6.172417,479.343140,6.197181,491.361816,"Dropout: 0.1, LR: 0.001, HD: [300, 300], Opt: SGD"


## Loading and Running the model again

In [39]:
with open('/kaggle/working/data_store_nnlm.pkl', 'rb') as f:
    data = pickle.load(f)

embeddings = data['embeddings']
vocab = data['vocab']
word_to_idx = data['word_to_idx']
encoded_train = data['encoded_train']
encoded_val = data['encoded_val']
encoded_test = data['encoded_test']

train_dataset = NGramDataset(encoded_train, embeddings)
val_dataset = NGramDataset(encoded_val, embeddings)
test_dataset = NGramDataset(encoded_test, embeddings)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

print("Data loaded successfully!")

Data loaded successfully!


In [40]:
model_new = NNLM(embeddings, [300, 300], 5, 0.1)

# Load the saved state_dict
model_new.load_state_dict(torch.load('/kaggle/working/2021101072_LM1.pt', weights_only=True))

model_new.eval()
model_new.to(device)

optimizer = optim.SGD(model_new.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()


In [41]:
loss, perplexity = test_model(model_new, val_loader, criterion)
print(f'\nVal Loss: {loss}')
print(f'Val Perplexity: {perplexity}')

100%|██████████| 2465/2465 [00:04<00:00, 588.90it/s]


Val Loss: 5.361447884441631
Val Perplexity: 213.0331573486328


In [42]:
loss, perplexity = test_model(model_new, test_loader, criterion)
print(f'\nTest Loss: {loss}')
print(f'Test Perplexity: {perplexity}')

100%|██████████| 1214/1214 [00:02<00:00, 605.10it/s]


Test Loss: 5.342875943348868
Test Perplexity: 209.11325073242188
